# Environment: Google Colab

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.0 MB/s eta 0:00:00


In [2]:
def refine_yolo_box(image, yolo_box):
    """
      Refines the YOLO bounding box with a SAFETY-FIRST approach.

      Major Fixes:
      1. Calculates 'Trapezoid Score'. If low, forces a simple Rotation (Plan B).
      2. Rejects contours that are too small or weirdly shaped to prevent 'Empty' images.
    """
    x1, y1, x2, y2 = map(int, yolo_box)
    h_img, w_img = image.shape[:2]

    # 1. Loose crop with conservative padding
    # To avoid accidental cropping of characters due to tight bounding box
    pad_x = int((x2 - x1) * 0.10)  # 10% horizontal padding
    pad_y = int((y2 - y1) * 0.15)  # 15% vertical padding

    crop_x1 = max(0, x1 - pad_x)
    crop_y1 = max(0, y1 - pad_y)
    crop_x2 = min(w_img, x2 + pad_x)
    crop_y2 = min(h_img, y2 + pad_y)

    loose_crop = image[crop_y1:crop_y2, crop_x1:crop_x2]

    if loose_crop.size == 0:
        return image[y1:y2, x1:x2], None, "Fallback (Empty)"

    loose_area = loose_crop.shape[0] * loose_crop.shape[1]

    # 2. Traditional Refinement
    gray = cv2.cvtColor(loose_crop, cv2.COLOR_BGR2GRAY)

    # Adaptive Threshold is safer for different lighting conditions
    binary = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        13, 3 # Block size 13, C=3 (Tuned for plates)
    )

    # Morphological operations to clean up noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    processed = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    processed = cv2.morphologyEx(processed, cv2.MORPH_DILATE, kernel, iterations=1)

    # Contour finding
    contours, _ = cv2.findContours(processed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    best_corners = None
    best_method = "Fallback"
    max_confidence_area = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)

        # Filter 1: Check size
        # Must be at least 10% of the crop, but not more than 95% (likely the frame itself)
        if area < (0.10 * loose_area) or area > (0.95 * loose_area):
            continue

        hull = cv2.convexHull(cnt)
        peri = cv2.arcLength(hull, True)

        # Plan A candidate: Polygon/Trapezoid approximation
        approx = cv2.approxPolyDP(hull, 0.04 * peri, True)

        # Plan B candidate: Rotated rectangle
        rect = cv2.minAreaRect(hull)
        box_rect = np.int32(cv2.boxPoints(rect))

        # Safety check
        # Assume plan B (rectangle) is the default best choice
        chosen_candidate = box_rect
        method = "Rotation (Plan B)"

        # Check if plan A (trapezoid) is valid and worth using
        if len(approx) == 4 and cv2.isContourConvex(approx):
            pts = approx.reshape(4, 2)

            # Calculate distortion score (how non-rectangular is it)
            rect_ordered = order_points(pts)
            (tl, tr, br, bl) = rect_ordered

            width_top = np.linalg.norm(tr - tl)
            width_bot = np.linalg.norm(br - bl)
            height_left = np.linalg.norm(tl - bl)
            height_right = np.linalg.norm(tr - br)

            # Score 1: parallelism (difference in width/height)
            width_diff = abs(width_top - width_bot) / max(width_top, width_bot)
            height_diff = abs(height_left - height_right) / max(height_left, height_right)

            # If the plate is mostly rectangular (diff < 0.20), use plan B
            # Use plan A if it is actually distorted (> 0.20).
            distortion_score = width_diff + height_diff

            if distortion_score > 0.20:
                chosen_candidate = pts
                method = "Perspective (Plan A)"
            else:
                # Basically a rectangle, don't risk warping and use plan B
                chosen_candidate = box_rect
                method = "Rotation (Plan B - Forced)"

        # Save the largest valid candidate found so far
        if area > max_confidence_area:
            max_confidence_area = area
            best_corners = chosen_candidate
            best_method = method

    # If no good contours, return None (trust YOLO)
    if best_corners is not None:
        # Check 1: Touching edge
        # If any corner is within 1 pixels of the image edge, assume it is risky
        h, w = loose_crop.shape[:2]
        x_min, y_min = np.min(best_corners, axis=0)
        x_max, y_max = np.max(best_corners, axis=0)

        is_touching_edge = (x_min < 1 or y_min < 1 or x_max > w-1 or y_max > h-1)

        if is_touching_edge:
            # Contour touch the edge of the crop and might cut the plate, so use fallback
            return loose_crop, None, "Fallback (Edge Touch Detected)"

        # Add extra space (fix for cropped characters) to ensure characters are not cropped
        safe_corners = scale_corners(best_corners, scale_factor=1.05)

        return loose_crop, safe_corners, best_method

    else:
        return loose_crop, None, "Fallback (YOLO)"

In [3]:
# Helper functions

def order_points(pts):
  """
    - Sorts coordinates in the order: top-left, top-right, bottom-right, bottom-left
    - For homography to work correctly
  """
  rect = np.zeros((4, 2), dtype="float32")

  # The top-left point will have the smallest sum, whereas the bottom-right point will have the largest sum
  s = pts.sum(axis=1)
  rect[0] = pts[np.argmin(s)] # top-left
  rect[2] = pts[np.argmax(s)] # bottom-right

  # The top-right point will have the smallest difference, whereas the bottom-left point will have the largest difference
  diff = np.diff(pts, axis=1)
  rect[1] = pts[np.argmin(diff)] # top-right
  rect[3] = pts[np.argmax(diff)] # bottom-left

  return rect

def scale_corners(pts, scale_factor=1.05):
  """
    - Expands the 4 corners outward from the center to give extra space
    - scale_factor=1.05 means expand by 5%
  """
  # Calculate center point
  center = np.mean(pts, axis=0)

  # Vector from center to each point
  vectors = pts - center

  # Scale coordinates
  expanded_pts = center + (vectors * scale_factor)
  return expanded_pts.astype(np.float32)

def four_point_transform(image, pts):
  """
    - Performs the perspective transformation.
    - Takes an image and 4 corners, returns the rectified image
  """
  rect = order_points(pts)
  (tl, tr, br, bl) = rect

  # Calculate the width of the new image
  # Maximum distance between bottom-right and bottom-left or top-right and top-left
  widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
  widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
  maxWidth = max(int(widthA), int(widthB))

  # Calculate the height of the new image
  # Maximum distance between top-right and bottom-right OR top-left and bottom-left
  heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
  heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
  maxHeight = max(int(heightA), int(heightB))

  # Construct the destination points (the perfect rectangle try to have)
  dst = np.array([
      [0, 0],
      [maxWidth - 1, 0],
      [maxWidth - 1, maxHeight - 1],
      [0, maxHeight - 1]], dtype="float32")

  # Get the perspective transform matrix (homography matrix)
  M = cv2.getPerspectiveTransform(rect, dst)

  # Warp the image
  warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
  return warped

In [4]:
from ultralytics import YOLO
import os
import glob
import cv2
import numpy as np

model = YOLO('/content/drive/MyDrive/YOLO12/best.pt')
input_dir = '/content/drive/MyDrive/Phase2_Results'
output_dir = '/content/drive/MyDrive/Phase3_4_Results'


def run_phase_3_4(subset_name):
    input_folder = os.path.join(input_dir, subset_name)
    output_folder = os.path.join(output_dir, subset_name)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    print(f"Processing {subset_name}")

    images = glob.glob(os.path.join(input_folder, "*.*"))
    count_warped = 0
    count_fallback = 0

    for img_path in images:
        if not img_path.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        img = cv2.imread(img_path)
        if img is None:
          continue

        # YOLO prediction
        results = model.predict(img, conf=0.5, verbose=False)

        if len(results[0].boxes) == 0:
            print(f"Skipped {os.path.basename(img_path)}: No plate found")
            continue

        # Get the box with highest confidence
        box = results[0].boxes[0].xyxy[0].cpu().numpy() # [x1, y1, x2, y2]

        # Refine and crop
        loose_crop, corners, status = refine_yolo_box(img, box)

        # Warping or not warping
        if corners is not None:
            try:
                # Plan A: Geometric Correction (apply perspective transform)
                final_plate = four_point_transform(loose_crop, corners)

                save_name = "WARPED_" + os.path.basename(img_path)
                count_warped += 1
            except Exception as e:
                # If warp calculation fails, revert to Plan B
                final_plate = loose_crop
                save_name = "FALLBACK_ERR_" + os.path.basename(img_path)
                if "Rotation (Plan B - Forced)" in status:
                  print(save_name)
                count_fallback += 1
        else:
            # Revert to Plan B if can't find a clean 4 point contour
            final_plate = loose_crop
            save_name = "FALLBACK_" + os.path.basename(img_path)
            if "Rotation (Plan B - Forced)" in status:
                  print(save_name)
            count_fallback += 1

        cv2.imwrite(os.path.join(output_folder, save_name), final_plate)

    print(f"Done. Warped (Corrected): {count_warped} | Fallback (Standard): {count_fallback}")

subsets = ["Subset_AC", "Subset_LE", "Subset_RP"]

for subset in subsets:
    run_phase_3_4(subset)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Processing Subset_AC
Done. Warped (Corrected): 6 | Fallback (Standard): 675
Processing Subset_LE
Done. Warped (Corrected): 49 | Fallback (Standard): 707
Processing Subset_RP
Done. Warped (Corrected): 39 | Fallback (Standard): 572
